In [1]:
import requests
import json

In [2]:
arr= requests.get('https://newsapi.org/v2/everything?'
       'q=Apple&'
       'from=2024-09-12&'
       'sortBy=popularity&'
       'apiKey=b647494796a7445082b34c0cdfd7deef')
arr

<Response [200]>

In [3]:
news_data =json.loads(arr.content)
news_data

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [4]:
articles = news_data['articles']

data = {
    'id': list(range(1, len(articles)+1)),
    'title': [article['title'] for article in articles],
    'author': [article['author'] if article['author'] else 'Unknown' for article in articles],
    'text': [article['description'] if article['description'] else 'No description available' for article in articles],
    'label': [1] * len(articles)  # You can assign labels based on your requirements
}

print(data)

{'id': [], 'title': [], 'author': [], 'text': [], 'label': []}


In [5]:
import pandas as pd

# Existing train.csv file
existing_file_path = 'train.csv'

# Create DataFrame
df = pd.DataFrame(data)

# Read existing CSV file
existing_df = pd.read_csv(existing_file_path)

# Append new data to the existing DataFrame
combined_df = pd.concat([existing_df, df], ignore_index=True)

# Save the combined DataFrame to train.csv
combined_df.to_csv(existing_file_path, index=False)

In [6]:
df=pd.read_csv("train.csv")

In [7]:
df.head()

,id,title,author,text,label,content
0,0.0,afbnn,ASFDGF,Ketan,0.0,NaN
1,0.0,afjefd,qwkfnwweg,Rohit,0.0,NaN
2,0.0,ket ydf,hellohgdgdh,air,0.0,NaN
3,0.0,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0.0,NaN
4,0.0,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0.0,NaN


In [8]:
df.describe()

,id,label,content
count,243.000000,243.000000,0.0
mean,10.308642,0.967078,NaN
std,5.948071,0.178800,NaN
min,0.000000,0.000000,NaN
25%,5.000000,1.000000,NaN
50%,10.000000,1.000000,NaN
75%,15.000000,1.000000,NaN
max,20.000000,1.000000,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       243 non-null    float64
 1   title    243 non-null    object 
 2   author   242 non-null    object 
 3   text     243 non-null    object 
 4   label    243 non-null    float64
 5   content  0 non-null      float64
dtypes: float64(3), object(3)
memory usage: 11.5+ KB


In [10]:
df.isnull().sum()

id           0
title        0
author       1
text         0
label        0
content    243
dtype: int64

In [11]:
df=df.fillna('')

In [12]:
df.isnull().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

In [13]:
df.columns

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

In [14]:
df=df.drop(['id', 'title', 'author'], axis=1)

In [15]:
df.head()

,text,label,content
0,Ketan,0.0,
1,Rohit,0.0,
2,air,0.0,
3,"PARIS — France chose an idealistic, traditi...",0.0,
4,Donald J. Trump is scheduled to make a highly ...,0.0,


In [16]:
from nltk.corpus import stopwords

In [17]:
from nltk.stem.porter import PorterStemmer

In [18]:
import re

In [19]:
port_stem=PorterStemmer()

In [20]:
port_stem

<PorterStemmer>

In [21]:
port_stem.stem("Hi thIs is chando * % %@@@")

'hi this is chando * % %@@@'

In [22]:
def stemming(content):
    con=re.sub('[^a-zA-Z]', ' ', content)
    con=con.lower()
    con=con.split()
    con=[port_stem.stem(word) for word in con if not word in stopwords.words('english')]
    con=' '.join(con)
    return con

In [23]:
stemming('Hi this is chando')

'hi chando'

In [24]:
df['text']= df['text'].apply(stemming)

In [25]:
x=df['text']

In [26]:
y=df['label']

In [27]:
y.shape

(243,)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x_train , x_test , y_train, y_test = train_test_split(x, y, test_size=0.20)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
vect=TfidfVectorizer()

In [32]:
x_train=vect.fit_transform(x_train)
x_test=vect.transform(x_test)

In [33]:
x_test.shape

(49, 1962)

In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
model=DecisionTreeClassifier()

In [36]:
model.fit(x_train, y_train)

DecisionTreeClassifier()

In [37]:
prediction=model.predict(x_test)

In [38]:
prediction

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [39]:
model.score(x_test, y_test)

1.0

In [40]:
import pickle

In [41]:
pickle.dump(vect, open('vector.pkl', 'wb'))

In [42]:
pickle.dump(model, open('model.pkl', 'wb'))

In [43]:
vector_form=pickle.load(open('vector.pkl', 'rb'))

In [44]:
load_model=pickle.load(open('model.pkl', 'rb'))

In [45]:
def fake_news(news):
    news=stemming(news)
    input_data=[news]
    vector_form1=vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction

In [46]:
val=fake_news("""king is dead""")
val[0]

0.0

In [47]:
if val==[1]:
    print('reliable')
else:
    print('unreliable')

unreliable
